## Data Cleaning

Steps for data cleaning 
 * load data sets
 * import additional data sets:
     * census population data from [Kaggle dataset](https://www.kaggle.com/peretzcohen/2019-census-us-population-data-by-state)
     * vaccine data? 
 * convert dates to datetime objects 
 * aggregate daily data to have one value per day 
 * restrict to contiguous US States and DC
 * handle NaNs? 
 * create proportions, daily differences, and rolling averages for Covid measures 
 * merge datasets together...? 
 
 * save cleaned data to `data/` folder

In [3]:
# pandas and numpy for working with data
import pandas as pd 
import numpy as np

# seaborn and matplotlib for visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# import daily reports by state 
daily_reports = pd.read_csv("../data/csse_covid_19_daily_reports_us.csv")
daily_reports.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
0,Alabama,US,2021-01-02 05:30:44,32.3182,-86.9023,365747,4872,202137.0,158738.0,1.0,7459.375895,1885216.0,NaN,1.332068,84000001.0,USA,38448.804196,NaN,NaN,NaN
1,Alaska,US,2021-01-02 05:30:44,61.3707,-152.4044,46986,206,7165.0,39615.0,2.0,6422.844801,1275750.0,NaN,0.438428,84000002.0,USA,174391.185778,NaN,NaN,NaN
2,American Samoa,US,2021-01-02 05:30:44,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,2140.0,NaN,NaN,16.0,ASM,3846.084722,NaN,NaN,NaN
3,Arizona,US,2021-01-02 05:30:44,33.7298,-111.4312,530267,9015,76934.0,444318.0,4.0,7285.171274,5155330.0,NaN,1.700087,84000004.0,USA,39551.860582,NaN,NaN,NaN
4,Arkansas,US,2021-01-02 05:30:44,34.9697,-92.3731,229442,3711,199247.0,26484.0,5.0,7602.945718,2051488.0,NaN,1.617402,84000005.0,USA,67979.497674,NaN,NaN,NaN


## Restricting to the Continental Unites States 

To analyze the transmission of COVID between regions in the US and consider the effects of neighboring regions we first restrict the data to regions in the Continental US. 

The daily reports data reports COVID numbers for all 50 states, plus the District of Columbia, several US territories (e.g. Guam, Puerto Rico) and two cruise ships (Diamond Princess and Grand Princess). The following cell selects only the data associated with the 48 continental US states and the District of Columbia (D.C.). 

In [5]:
# show all provinces/states included in data set
daily_reports['Province_State'].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Recovered'], dtype=object)

In [22]:
# the following are the provinces/states to exclude 
excluded_regions = ['Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands', 'American Samoa', # territories
                    'Diamond Princess', 'Grand Princess', 'Guam', 'Recovered', # other 
                    'Alaska', 'Hawaii'] # non continuous states
# this creates a list of the continental US states and DC in the list 
continguous_US = list(set(list(daily_reports['Province_State'].unique())) - set(excluded_regions))
assert len(continguous_US) == 49 # 50 states + DC - Hawaii, Alaska

# create a dataframe with only data for these continental regions 
cont_daily_reports = daily_reports[daily_reports['Province_State'].isin(continguous_US)]
assert (set(cont_daily_reports['Province_State'].unique()) == set(continguous_US))

## Examine Columns

In [23]:
#check data type for date
cont_daily_reports.dtypes

Province_State           object
Country_Region           object
Last_Update              object
Lat                     float64
Long_                   float64
Confirmed                 int64
Deaths                    int64
Recovered               float64
Active                  float64
FIPS                    float64
Incident_Rate           float64
Total_Test_Results      float64
People_Hospitalized     float64
Case_Fatality_Ratio     float64
UID                     float64
ISO3                     object
Testing_Rate            float64
Hospitalization_Rate    float64
People_Tested           float64
Mortality_Rate          float64
dtype: object

The `Last_Update` column contains a date but is stored as an object, we convert this to DateTime to work with the data as a date. The documentation states that this value represents a date and time in UTC. 

In [24]:
cont_daily_reports.loc[:,'Last_Update'] = pd.to_datetime(cont_daily_reports['Last_Update'], utc=True)

In [25]:
## Country_Region
display(cont_daily_reports['Country_Region'].value_counts())

display(cont_daily_reports['ISO3'].value_counts())

US    17346
Name: Country_Region, dtype: int64

USA    17346
Name: ISO3, dtype: int64

Since all values are the same in this dataset for `Country_Region` and `ISO3`, we can drop this column. 

**FINISH ANALYSIS OF OTHER COLUMNS HERE** 

create lists for numerics of interest (`covid_metrics`) 

do we need UID? how many NANs are there in each column?? 

In [26]:
cont_daily_reports = cont_daily_reports.drop(columns=['Country_Region','ISO3'])
cont_daily_reports.head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
0,Alabama,2021-01-02 05:30:44+00:00,32.3182,-86.9023,365747,4872,202137.0,158738.0,1.0,7459.375895,1885216.0,NaN,1.332068,84000001.0,38448.804196,NaN,NaN,NaN
3,Arizona,2021-01-02 05:30:44+00:00,33.7298,-111.4312,530267,9015,76934.0,444318.0,4.0,7285.171274,5155330.0,NaN,1.700087,84000004.0,39551.860582,NaN,NaN,NaN
4,Arkansas,2021-01-02 05:30:44+00:00,34.9697,-92.3731,229442,3711,199247.0,26484.0,5.0,7602.945718,2051488.0,NaN,1.617402,84000005.0,67979.497674,NaN,NaN,NaN
5,California,2021-01-02 05:30:44+00:00,36.1162,-119.6816,2365024,26363,NaN,2309915.0,6.0,5912.603297,33058311.0,NaN,1.125160,84000006.0,83666.036710,NaN,NaN,NaN
6,Colorado,2021-01-02 05:30:44+00:00,39.0598,-105.3111,337161,4873,18102.0,314186.0,8.0,5854.774381,4444206.0,NaN,1.445304,84000008.0,77173.289416,NaN,NaN,NaN


## Uniformizing Timeseries Differences

For analyzing the spread of COVID in time we first want to ensure that the data are regularly distributed in time. In the next few cells we will see that for the most part, there is one entry per Province_State.

Since this data is reported daily, we expect to see one row per Province_State per day. Indeed looking at a few rows from California seems to align with this. 

In [30]:
cont_daily_reports[cont_daily_reports['Province_State']=='California'].sort_values('Last_Update').head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
5224,California,2020-04-12 23:18:15+00:00,36.1162,-119.6816,22805,640,NaN,22155.0,6.0,58.137726,NaN,5234.0,NaN,84000006.0,485.423869,22.961176,190328.0,2.812020
5284,California,2020-04-13 23:07:54+00:00,36.1162,-119.6816,23946,714,NaN,23217.0,6.0,61.035048,NaN,3015.0,NaN,84000006.0,486.836823,12.598721,190882.0,2.987756
5343,California,2020-04-14 23:33:31+00:00,36.1162,-119.6816,25373,767,NaN,24589.0,6.0,64.669453,NaN,5163.0,NaN,84000006.0,515.723328,20.362044,202208.0,3.028869
5402,California,2020-04-15 22:56:51+00:00,36.1162,-119.6816,26699,860,NaN,25826.0,6.0,68.061564,NaN,5163.0,NaN,84000006.0,552.138790,19.347223,216486.0,3.226411
5461,California,2020-04-16 23:30:51+00:00,36.1162,-119.6816,27687,956,NaN,26721.0,6.0,70.589069,NaN,5031.0,NaN,84000006.0,628.433237,18.177548,246400.0,3.457745


However if we extract just the date from the `Last_Update` and look at the differences between sequential rows (once ordered by date), we see there are many cases of a 2 day jump and 1 instance of two reports on a single day. 

In [31]:
# extract date only from UTC datetime
cont_daily_reports['UTC_Date_Last_Update'] = cont_daily_reports['Last_Update'].dt.date

In [34]:
def compute_day_diffs(data, date_col):
    '''
    This function takes a dataframe with a date column and returns the 
    dataframe with an additional 
    '''
    # sort by the given date column
    sorted_data = data.sort_values(date_col)
    # take sequential differences and relabel column to 'Days_Diff', note first entry will be NaT
    day_diffs = sorted_data[[date_col]].diff().rename(columns={date_col:'Days_Diff'})
    # merge back into initial data 
    sorted_data = sorted_data.merge(day_diffs, how="inner", left_index=True, right_index=True)
    return sorted_data

def extract_unusual_daily_diffs(data, date_col):
    daily_diff_data = compute_day_diffs(data, date_col)
    # select only rows that are not of daily frequency or the first row (always NaT/Na)
    unusual_date_diffs = sorted_data[(sorted_data['Days_Diff']!=pd.Timedelta('1D')) & (~ sorted_data['Days_Diff'].isna())]
    return unusual_date_diffs

unusual_date_diff_data = cont_daily_reports.groupby('Province_State').apply(extract_unusual_day_diffs, 
                                                                            date_col='UTC_Date_Last_Update')
unusual_date_diff_data = unusual_date_diff_data.set_index(unusual_date_diff_data.index.droplevel('Province_State'))
unusual_date_diff_data[['Province_State','Last_Update','UTC_Date_Last_Update','Days_Diff']].head()


,Province_State,Last_Update,UTC_Date_Last_Update,Days_Diff
5869,Alabama,2020-04-24 03:33:00+00:00,2020-04-24,"2 days, 0:00:00"
5872,Arizona,2020-04-24 03:33:00+00:00,2020-04-24,"2 days, 0:00:00"
5873,Arkansas,2020-04-24 03:33:00+00:00,2020-04-24,"2 days, 0:00:00"
5874,California,2020-04-24 03:33:00+00:00,2020-04-24,"2 days, 0:00:00"
5875,Colorado,2020-04-24 03:33:00+00:00,2020-04-24,"2 days, 0:00:00"


In [35]:
unusual_date_diff_data[['Days_Diff']].value_counts()

Days_Diff
2 days       50
0 days        1
dtype: int64

From the tables above we see there are 50 2 day jumps and also a single 0 day jump (2 reports on the same day). The next few cells will investigate this further. Using the dates from the unusual rows we can look at the data around that time to see what is going on. 

In [37]:
def data_between_dates(data, date_col, start_date='2020-01-01', end_date='2021-04-03'):
    '''Takes a dataframe and a date column and returns rows where the date column lies between given dates'''
    return data[(data[date_col] >= pd.to_datetime(start_date)) & (data[date_col] <= pd.to_datetime(end_date))]

In [40]:
data_between_dates(cont_daily_reports[cont_daily_reports['Province_State'].isin(['California','New York'])],
                   date_col = 'UTC_Date_Last_Update', start_date = '2020-04-22', end_date = '2020-04-25').sort_values('Province_State')

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate,UTC_Date_Last_Update
5815,California,2020-04-22 23:40:26+00:00,36.1162,-119.6816,37731,1421,NaN,35923.0,6.0,95.244362,NaN,4984.0,NaN,84000006.0,1186.797699,13.346187,465327.0,3.805163,2020-04-22
5874,California,2020-04-24 03:33:00+00:00,36.1162,-119.6816,39946,1533,NaN,38028.0,6.0,100.898731,NaN,4929.0,NaN,84000006.0,1229.568906,12.459240,482097.0,3.875028,2020-04-24
5933,California,2020-04-25 06:32:46+00:00,36.1162,-119.6816,41803,1621,NaN,39734.0,6.0,105.474255,NaN,4880.0,NaN,84000006.0,1260.368256,11.800266,494173.0,3.919720,2020-04-25
5846,New York,2020-04-22 23:40:26+00:00,42.1657,-74.9481,263292,19413,23887.0,243879.0,36.0,1561.542077,NaN,57103.0,NaN,84000036.0,3973.554395,21.688088,669982.0,7.373183,2020-04-22
5905,New York,2020-04-24 03:33:00+00:00,42.1657,-74.9481,263460,20973,23887.0,242487.0,36.0,1562.538458,NaN,57103.0,NaN,84000036.0,4127.388459,21.674258,695920.0,7.960601,2020-04-24
5964,New York,2020-04-25 06:32:46+00:00,42.1657,-74.9481,271590,21411,23887.0,250179.0,36.0,1610.756167,NaN,57103.0,NaN,84000036.0,4333.402032,21.025443,730656.0,7.883575,2020-04-25


We see above that the times for the reports changes significantly from around 23:30 UTC (almost midnight) to 3:30-6:30 UTC (early morning) between April 22, 2020 and April 24, 2020. Reviewing the [documentation](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data#time-series-summary-csse_covid_19_time_series), there is the following note: 

    Files on and after April 23, once per day between 03:30 and 04:00 UTC.
    Files from February 2 to April 22: once per day around 23:59 UTC.
    
Since the times are in UTC, but the data is being reported for US states, an early morning UTC time actually corresponds to a time _the previous day_ in the US. So the missing April 23rd data is actually the April 24th data. Each province/state is probably encountering the same time change issue, accounting for 49 (number of distinct provinces/states in dataframe) of the 51 unusual time differences. We can fix this for all regions and then check what differences remain. 

In [46]:
# function shifts datetime up one day for entries after April 22, 2020
def add_day_after_date(datetime):    
    if datetime > pd.to_datetime('2020-04-22'):
        return datetime - pd.DateOffset(1)
    return datetime

cont_daily_reports['UTC_Adjusted_Date'] = cont_daily_reports['UTC_Date_Last_Update'].apply(add_day_after_date)

In [47]:
remaining_unusual_date_diff_data = cont_daily_reports.groupby('Province_State').apply(extract_unusual_day_diffs, 
                                                                            date_col='UTC_Adjusted_Date')
remaining_unusual_date_diff_data = remaining_unusual_date_diff_data.set_index(remaining_unusual_date_diff_data.index.droplevel('Province_State'))
remaining_unusual_date_diff_data[['Province_State','Last_Update','UTC_Date_Last_Update','UTC_Adjusted_Date','Days_Diff']].head()

,Province_State,Last_Update,UTC_Date_Last_Update,UTC_Adjusted_Date,Days_Diff
5290,Florida,2020-04-14 00:42:00+00:00,2020-04-14,2020-04-14,2 days
5349,Florida,2020-04-14 23:33:31+00:00,2020-04-14,2020-04-14,0 days


The remaining two issues are with Florida, which has at least two entries on April 14, 2020. Let's again examine the entries close to this. 

In [49]:
data_between_dates(cont_daily_reports[cont_daily_reports['Province_State']=='Florida'],
                   date_col = 'UTC_Adjusted_Date', start_date = '2020-04-12', end_date = '2020-04-15').sort_values('Province_State')

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate,UTC_Date_Last_Update,UTC_Adjusted_Date
5230,Florida,2020-04-12 23:18:15+00:00,27.7663,-81.6868,19895,461,NaN,19434.0,12.0,93.700227,NaN,2772.0,NaN,84000012.0,860.718651,13.933149,182753.0,2.317165,2020-04-12,2020-04-12
5290,Florida,2020-04-14 00:42:00+00:00,27.7663,-81.6868,21019,499,NaN,122520.0,12.0,579.387193,NaN,2833.0,NaN,84000012.0,924.083459,2.302896,196207.0,0.405628,2020-04-14,2020-04-14
5349,Florida,2020-04-14 23:33:31+00:00,27.7663,-81.6868,21628,571,NaN,21057.0,12.0,101.862202,NaN,3060.0,NaN,84000012.0,956.924459,14.148326,203180.0,2.640096,2020-04-14,2020-04-14
5408,Florida,2020-04-15 22:56:51+00:00,27.7663,-81.6868,22511,596,NaN,21915.0,12.0,106.020900,NaN,3344.0,NaN,84000012.0,1005.571337,14.854960,213509.0,2.647595,2020-04-15,2020-04-15


Both unusual time differences are coming from the fact that row 5290 has a date of 2020-04-14 instead of 2020-04-13 as we might expect. Examining the time for this entry, it is just after midnight. Since entries before April 23 are expected to be logged close to midnight for that day, this appears to be a late entry, reporting data for Florida for April 13, 2020 but labeled for the next day in the dataset. To fix this, we can manually alter the day for this Florida entry to match the (likely) intended day the data represents. 

In [50]:
# manually adjust Florida's late date entry 
cont_daily_reports.loc[5290,'UTC_Adjusted_Date'] = pd.to_datetime('2020-04-13')

We can now check that there are no unusual time jumps remaining. Passing the following check verifies that all rows are 'daily', i.e. there is a 1 day gap between all sequential rows per state/province. 

In [53]:
remaining_unusual_date_diff_data = cont_daily_reports.groupby('Province_State').apply(extract_unusual_day_diffs, 
                                                                            date_col='UTC_Adjusted_Date')
remaining_unusual_date_diff_data = remaining_unusual_date_diff_data.set_index(remaining_unusual_date_diff_data.index.droplevel('Province_State'))
assert remaining_unusual_date_diff_data.shape[0] == 0

Now that we have created a date column to represent the local date for each row, we can remove our intermediate date columns and relabel our final column. 

In [55]:
cont_daily_reports = cont_daily_reports.drop(columns = ['Last_Update','UTC_Date_Last_Update'])
cont_daily_reports = cont_daily_reports.rename(columns={'UTC_Adjusted_Date':'Date'})
cont_daily_reports.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate,Date
0,Alabama,32.3182,-86.9023,365747,4872,202137.0,158738.0,1.0,7459.375895,1885216.0,NaN,1.332068,84000001.0,38448.804196,NaN,NaN,NaN,2021-01-01
3,Arizona,33.7298,-111.4312,530267,9015,76934.0,444318.0,4.0,7285.171274,5155330.0,NaN,1.700087,84000004.0,39551.860582,NaN,NaN,NaN,2021-01-01
4,Arkansas,34.9697,-92.3731,229442,3711,199247.0,26484.0,5.0,7602.945718,2051488.0,NaN,1.617402,84000005.0,67979.497674,NaN,NaN,NaN,2021-01-01
5,California,36.1162,-119.6816,2365024,26363,NaN,2309915.0,6.0,5912.603297,33058311.0,NaN,1.125160,84000006.0,83666.036710,NaN,NaN,NaN,2021-01-01
6,Colorado,39.0598,-105.3111,337161,4873,18102.0,314186.0,8.0,5854.774381,4444206.0,NaN,1.445304,84000008.0,77173.289416,NaN,NaN,NaN,2021-01-01


## Adding Data

## Sorting Columns by Type 

In [ ]:
covid_metrics = ['Confirmed', 'Deaths', 'Recovered', 'Active', 'Total_Test_Results', 'People_Hospitalized']


In [ ]:
# create lists with groups of covid metrics (split by processing type)
covid_metrics_props = [metric + "_prop" for metric in covid_metrics]
covid_metrics_diffs = [metric + "_diff" for metric in covid_metrics]
covid_metrics_prop_diffs = [metric + "_prop_diff" for metric in covid_metrics]
covid_metrics_prop_diffs_rolling_avg_7D = [metric + "_prop_diff_rolling_avg_7D" for metric in covid_metrics]
covid_metrics_prop_diffs_rolling_avg_7D

In [ ]:
# import data state population data from https://www.kaggle.com/peretzcohen/2019-census-us-population-data-by-state
population_data = pd.read_csv('../data/2019_Census_US_Population_Data_By_State_Lat_Long.csv')
population_data.rename(columns={'STATE':'State', 'POPESTIMATE2019': 'Total Pop'}, inplace=True)
population_data.drop(columns=['lat','long'], inplace=True)
population_data.head()

In [ ]:
# merge population data into daily_counts dataframe 
daily_reports = daily_reports.merge(population_data, how='left', left_on='Province_State', right_on='State')

In [ ]:
# define helper functions for pipeline of data processing 
def load_state_data(state):
    return daily_reports[daily_reports["Province_State"]==state]

def create_proportion(data, col_name, total_col, output_name=False):
    if not output_name:
        output_name = col_name + '_prop'
    data_copy = data.copy()
    data_copy.loc[:,output_name] = data[col_name] / data[total_col]
    return data_copy 

def create_proportions(data, cols, total_col):
    for col_name in cols:
        data = create_proportion(data, col_name, total_col)
        print('Created proportions for {}'.format(col_name))
    return data 


def create_diff(data, col_name, output_name=False):
    if not output_name:
        output_name = col_name + '_diff'
    sorted_data = data.sort_values(by="Date_Last_Update")
    sorted_data.loc[:,output_name] = sorted_data.loc[:,col_name].diff()
    return sorted_data

def create_diffs(data, cols):
    for col_name in cols:
        data = create_diff(data, col_name)
        print('Created diffs for {}'.format(col_name))
    return data

def create_rolling_average_by_time(data, col_name,  window_offset, 
                                   output_name = False, date_col="Date_Last_Update"):
    if not output_name:
        output_name = col_name + '_rolling_avg_' + str(window_offset)
    temp_df = (data[[date_col,col_name]]
           .set_index(date_col)
           .sort_index()
           .rolling(window_offset).mean().reset_index()
           .rename(columns={col_name:output_name})
          )
    #requires day col to be unique! 
    data = data.merge(temp_df, how="left", left_on=date_col, right_on=date_col)
    return data

def create_rolling_averages_by_time(data, cols, window_offset):
    for col_name in cols:             
        print('Creating rolling avgs for {}'.format(col_name))
        data = create_rolling_average_by_time(data, col_name, window_offset)
    return data

def apply_pipeline(data):
    return (data.pipe(create_proportions, 
                      cols = covid_metrics, 
                      total_col = 'Total Pop')
                .pipe(create_diffs, cols=covid_metrics)
                .pipe(create_diffs, cols=covid_metrics_props)
                .pipe(create_rolling_averages_by_time, 
                    cols = covid_metrics_prop_diffs,
                    window_offset = '7D')
                .pipe(create_rolling_averages_by_time, 
                    cols = covid_metrics_diffs,
                    window_offset = '7D')
            )

In [ ]:
def plot_states_over_time(state_names, feature_col, time_col="Date_Last_Update"):
    for state in state_names:
        sns.lineplot(data=state_reports[state], x=time_col, y=feature_col);
    plt.legend(labels=state_names);

In [ ]:
florida = create_proportion(load_state_data('Florida'), 'Total_Test_Results', 'Total Pop')
florida = create_diff(florida, 'Total_Test_Results_prop')
florida = create_rolling_average_by_time(florida, 'Total_Test_Results_prop_diff', '7D')
florida.head()

In [ ]:
florida = apply_pipeline(load_state_data('Florida'))

In [ ]:
# read in states into dictionary and apply pipeline to each 
state_reports = {}
if 'Florida' in continguous_US:
    continguous_US.remove('Florida')
for state in continguous_US:
    print(state)
    # the following line is getting hung up on create rolling avgs for Test_Results_prop_diff for Florida... 
    state_reports[state] = apply_pipeline(load_state_data(state))

In [ ]:
# read in states into dictionary and apply pipeline to each 
state_reports = {}
for state in ['California','Oregon','Nevada','Arizona']:
    print(state)
    state_reports[state] = apply_pipeline(load_state_data(state))

In [ ]:
state_reports['California'].head()

In [ ]:
plot_states_over_time(['California','Oregon','Nevada','Arizona'], 'Confirmed_prop_diff_rolling_avg_7D')